In [2]:
import trimesh
import numpy as np
import k3d

import struct
import open3d as o3d

from random import randint
from sklearn.cluster import DBSCAN

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# Load the PLY mesh
mesh = trimesh.load_mesh("/scratch/rhm4nj/cral/cral-ginn/Replica-Dataset/ReplicaSDK/apartment_1/mesh.ply")
vertices = np.array(mesh.vertices)  # (N, 3) array of vertex positions
faces = np.array(mesh.faces)  # (M, 3) array of triangle indices

print(f"Loaded mesh with {vertices.shape[0]} vertices and {faces.shape[0]} faces.")


Loaded mesh with 1909992 vertices and 3817138 faces.


In [4]:
def downsample_random(points, K):
    indices = np.random.choice(points.shape[0], K, replace=False)
    return points[indices]

plot = k3d.plot()

point_cloud = k3d.points(downsample_random(vertices, 500_000), point_size=0.01, color=0xFF0000)  # Red color
plot += point_cloud

# mesh_plot = k3d.mesh(vertices, faces, color=0x00FF00)  # Green color
# plot += mesh_plot

plot.display()

/scratch/rhm4nj/.conda/ginn_env11/lib/python3.11/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

In [32]:
points = vertices

def ransac_plane_segmentation(point_cloud, distance_threshold=0.01, max_iterations=1000):
    o3d_cloud = o3d.geometry.PointCloud()
    o3d_cloud.points = o3d.utility.Vector3dVector(point_cloud)

    planes = []
    plane_models = []
    while True:
        plane_model, inliers = o3d_cloud.segment_plane(
            distance_threshold=distance_threshold,
            ransac_n=5,
            num_iterations=max_iterations
        )

        if len(inliers) < 50:  # Adjust this threshold as needed
            break

        plane_points = np.asarray(o3d_cloud.points)[inliers]
        planes.append(plane_points)
        plane_models.append(plane_model)  # Store the plane model (ax + by + cz + d = 0)

        # Remove inliers from the point cloud
        o3d_cloud = o3d_cloud.select_by_index(inliers, invert=True)

    return planes, plane_models

def remove_subset_points(cloud1, cloud2):
    matches = np.isin(cloud1[:, None], cloud2).all(axis=2).any(axis=1)
    filtered_cloud1 = cloud1[~matches]
    return filtered_cloud1

def get_planes_parallel_to_floor(planes, plane_models, floor_model, tolerance=0.2):
    # Extract the normal vector of the floor plane
    floor_normal = np.array(floor_model[:3])

    # Find parallel planes
    parallel_planes = []
    for i, model in enumerate(plane_models):
        plane_normal = np.array(model[:3])
        cos_theta = np.dot(plane_normal, floor_normal) / (np.linalg.norm(plane_normal) * np.linalg.norm(floor_normal))

        # Check if planes are parallel within the tolerance
        if abs(abs(cos_theta) - 1) < tolerance:
            parallel_planes.append(planes[i])

    return parallel_planes

def segment_planes(point_cloud):
    def compute_curvature(pcd, k=20):
        """ Estimate curvature to filter out non-planar regions """
        pcd.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(k))
        curvatures = np.linalg.norm(np.asarray(pcd.normals), axis=1)
        return curvatures
    
    def region_growing_clustering(pcd, eps=0.05, min_cluster_size=500):
        """ Cluster the plane points to extract only contiguous flat regions """
        labels = np.array(pcd.cluster_dbscan(eps=eps, min_points=min_cluster_size, print_progress=False))
        unique_labels = np.unique(labels)
        
        clusters = []
        for label in unique_labels:
            if label == -1:  # Ignore noise
                continue
            cluster_pcd = pcd.select_by_index(np.where(labels == label)[0])
            clusters.append(cluster_pcd)
        
        return clusters
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_cloud)
    
    curvature = compute_curvature(pcd)
    flat_indices = np.where(curvature < 1)[0]  # Adjust threshold
    flat_pcd = pcd.select_by_index(flat_indices)
    
    planes = []
    plane_points = []
    remaining_pcd = flat_pcd
    
    while len(remaining_pcd.points) > 500:  # Stop when too few points remain
        plane_model, inliers = remaining_pcd.segment_plane(distance_threshold=0.09,
                                                           ransac_n=3,
                                                           num_iterations=1000)
        
        plane_pcd = remaining_pcd.select_by_index(inliers)
        clustered_planes = region_growing_clustering(plane_pcd, eps=0.75, min_cluster_size=700)
        # print(plane_pcd, clustered_planes)
        
        for cluster in clustered_planes:
            planes.append(cluster)
            plane_points.append(np.asarray(cluster.points))
        
        remaining_pcd = remaining_pcd.select_by_index(inliers, invert=True)
        
    return plane_points, planes

print("Raw points shape:", points.shape)
max_points = 500_000

if points.shape[0] > max_points:
    points = downsample_random(points, max_points)
    print("Downsample", max_points)

# print("RANSAC")
# plane_clusters, plane_models = ransac_plane_segmentation(points, distance_threshold=0.03, max_iterations=1000)
# floor = plane_clusters[0]
# floor_model = plane_models[0]  # Assume the first detected plane is the floor

# parallel_planes = get_planes_parallel_to_floor(plane_clusters, plane_models, floor_model)
# parallel_planes.append(floor)
# print("N parallel:", len(parallel_planes))

plane_points, planes = segment_planes(points)

Raw points shape: (1909992, 3)
Downsample 500000


In [33]:
# clusters_sorted = sorted(clusters, key=lambda x: x.shape[0], reverse=True)

print(len(plane_points))
plot = k3d.plot()
n = 5

# color = randint(0, 0xFFFFFF)
# plot += k3d.points(
#     positions=plane_points, 
#     point_size=0.1,
#     color=color  # Apply the random color
# )

plane_points = sorted(plane_points, key=lambda x: x.shape[0], reverse=True)
print(plane_points)

for i, cluster in enumerate(plane_points[:n]):
    color = randint(0, 0xFFFFFF)

    r = (color >> 16) & 0xFF  # Extract the red component
    g = (color >> 8) & 0xFF   # Extract the green component
    b = color & 0xFF          # Extract the blue component
    rgb = (r, g, b)

    print(i, rgb)
    
    plot += k3d.points(
        positions=cluster.astype(np.float32), 
        point_size=0.05,
        color=color  # Apply the random color
    )

plot.display()

15
[array([[ 6.88550901,  4.50611925,  0.98404688],
       [-1.03884006, -0.68322343,  1.12217116],
       [ 5.06803608,  4.56223011,  0.99797118],
       ...,
       [ 5.93379688,  1.30822921,  1.046808  ],
       [ 4.79471159,  3.01433468,  1.03727984],
       [ 2.96850729,  2.40629268,  1.07144475]], shape=(42742, 3)), array([[ 1.56331098,  0.12689532, -1.62484145],
       [ 7.09966803,  6.8258357 , -1.79109514],
       [ 7.22564793,  3.17730904, -1.71597087],
       ...,
       [ 8.00860691,  2.68246198, -1.71098387],
       [ 3.51489019,  4.6831193 , -1.71295094],
       [ 5.04611969, -0.23202941, -1.65144134]], shape=(41956, 3)), array([[ 1.99925804,  2.88870645, -0.03965694],
       [ 2.02875662,  3.66734624,  0.9132002 ],
       [ 2.04883361,  5.41215181, -1.16312313],
       ...,
       [ 2.07527471,  0.97693455,  0.85300469],
       [ 2.04109097,  6.34492874,  0.24310727],
       [ 2.05870342,  1.65716815, -0.49731424]], shape=(17716, 3)), array([[ 4.50174427,  3.01168919,  0

Output()

In [30]:
def dbscan_cluster(point_cloud, eps=0.5, min_samples=5):
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(point_cloud)

    labels = db.labels_

    unique_labels = set(labels)
    if -1 in unique_labels:
        unique_labels.remove(-1)

    clusters = []
    for label in unique_labels:
        cluster_points = point_cloud[labels == label]
        clusters.append(cluster_points)

    return clusters

print("Cleaning")
cloud = points.copy()
print(cloud.shape)
for plane in plane_points:
    cloud = remove_subset_points(cloud, plane)
    
print(cloud.shape)

print("Clustering")
cloud = points
clusters = dbscan_cluster(cloud, eps=0.5, min_samples=200)
print(len(clusters))

Cleaning
(500000, 3)
(354444, 3)
Clustering
1


In [34]:
clusters_sorted = sorted(clusters, key=lambda x: x.shape[0], reverse=True)
print(len(clusters_sorted))

plot = k3d.plot()
n = 5

color = randint(0, 0xFFFFFF)
plot += k3d.points(
    positions=cloud, 
    point_size=0.05,
    color=color  # Apply the random color
)

# for i, cluster in enumerate(clusters_sorted[:n]):
#     color = randint(0, 0xFFFFFF)

#     r = (color >> 16) & 0xFF  # Extract the red component
#     g = (color >> 8) & 0xFF   # Extract the green component
#     b = color & 0xFF          # Extract the blue component
#     rgb = (r, g, b)
#     print(i, rgb)
    
#     plot += k3d.points(
#         positions=cluster.astype(np.float32), 
#         point_size=0.05,
#         color=color  # Apply the random color
#     )

plot.display()

1


Output()

In [ ]:
points = clusters_sorted[0]
scale_factor = (points - points.min(axis=0)) / (points.max(axis=0) - points.min(axis=0))
points = (points - points.min(axis=0)) / (points.max(axis=0) - points.min(axis=0))

max_iters = 3
for i in tqdm(range(max_iters), desc="iter"):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)

    pcd, ind = pcd.remove_statistical_outlier(nb_neighbors=30, std_ratio=0.5)
    pcd, ind = pcd.remove_radius_outlier(nb_points=30, radius=0.15)

    alpha = 10
    alpha_shape = alphashape.alphashape(points, alpha)

    if isinstance(alpha_shape, trimesh.Trimesh):
        is_closed = alpha_shape.is_watertight
        if is_closed:
            print("The alpha shape is closed (watertight).")
        else:
            print("The alpha shape is not closed (not watertight).")
    else:
        print("The alpha shape is not a 3D mesh.")

    points = np.concatenate([points, (alpha_shape.vertices)])

plot = k3d.plot()
point_size = 0.005
plot += k3d.points(positions=points.astype(np.float32), point_size=point_size, color=0x0000ff)
plot += k3d.points(positions=(alpha_shape.vertices).astype(np.float32), point_size=point_size, color=0xff00ff)

vertices = np.array(alpha_shape.vertices)
faces = np.array(alpha_shape.faces)
plot += k3d.mesh(vertices=vertices.astype(np.float32),
                    indices=faces.astype(np.uint32),
                    color=0xff0000,
                    opacity=0.5)  # Set opacity for visualization

plot.camera_position = [500, 500, 500]  # Move the camera further back for a zoomed-out view
plot.camera_target = [0, 0, 0]          # Set the target point (center of the view)
plot.camera_up = [0, 1, 0]              # Set the camera's up direction

plot.display()


NameError: name 'tqdm' is not defined